In [76]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import joblib

: 

In [77]:
# Load the dataset
df = pd.read_csv('coconut_irrigation_data_with_water_level_8.csv')

In [ ]:
df.head(2)

In [79]:
# Convert Water Level (liters) to range categories
def categorize_water_level(water_level):
    if 50 <= water_level <= 100:
        return 1  # High water need
    elif 30 <= water_level < 50:
        return 2  # Moderate water need
    elif 10 <= water_level < 30:
        return 3  # Low water need
    else:
        return 0  # No water needed or out of defined range

In [80]:
# Apply the categorization function to the Water Level column
df['Water Level Category'] = df['Water Level (liters)'].apply(categorize_water_level)

In [81]:
Water_Level_Category_counts = df['Water Level Category'].value_counts()

In [83]:
# Prepare the features and target variable
X = df.drop(['Date','Rainfall (mm)' ,'Irrigation Status (0/1)' ,'Water Level (liters)','Water Level Category'], axis=1)  # Features
y = df['Water Level Category']                           # Target variable

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [84]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='auto')
X_resampled, y_resampled = smote.fit_resample(X, y)

In [85]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [86]:
# Function to evaluate and print model results
def evaluate_model(model):
    model.fit(X_train, y_train)  # Train the model
    y_pred = model.predict(X_test)  # Make predictions

    # Print evaluation metrics
    print(f"Model: {model.__class__.__name__}")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))

    # Plot confusion matrix
    plt.figure(figsize=(6, 4))
    sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues', 
                 xticklabels=['0', '1','2','3'], 
                 yticklabels=['0', '1','2','3'])
    plt.title(f'Confusion Matrix - {model.__class__.__name__}')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()
    
    # Export the model
    model_filename = f"{model.__class__.__name__.lower()}water_model.pkl"
    joblib.dump(model, model_filename)
    print(f"Model exported as: {model_filename}")

In [ ]:
# Create and evaluate models
models = [
    RandomForestClassifier( random_state=42),
    DecisionTreeClassifier(random_state=42),
    LogisticRegression(max_iter=200, random_state=42)
]

for model in models:
    evaluate_model(model)


In [88]:
# Load the model and make predictions on new data
def load_model_and_predict(model_filename, new_data):
    # Load the model
    model = joblib.load(model_filename)
    
    # Make predictions
    predictions = model.predict(new_data)
    return predictions


In [ ]:
df.head(2)

In [98]:
# Generate new data
new_data = pd.DataFrame({
    'Soil Moisture (10 cm) (%)': [14.477],
    'Soil Moisture (20 cm) (%)': [24.64],
    'Soil Moisture (30 cm) (%)': [36.25],
    'Plant Age (years)': [5],
    'Temperature (°C)': [28.3],
    'Humidity (%)': [72.52],
    'Rain Status (0/1)': [0], 
})

In [ ]:
predictions = load_model_and_predict('randomforestclassifierwater_model.pkl', new_data)
predictions

In [96]:
# Convert range categories to Water Level (liters) range as a string
def get_water_level_range_string_from_category(category):
    if category == 1:
        return "50 - 100 (High water need)"  # High water need
    elif category == 2:
        return "30 - 50 (Moderate water need)"  # Moderate water need
    elif category == 3:
        return "10 - 30 (Low water need)"  # Low water need
    elif category == 0:
        return "0 - 0 (No water needed)"  # No water needed
    else:
        raise ValueError("Invalid category. Must be 0, 1, 2, or 3.")


In [ ]:
prediction = get_water_level_range_string_from_category(predictions[0])
prediction